In [14]:
# !pip install -q noisereduce
# !pip install -q git+https://github.com/TenzinGayche/num2tib.git

In [3]:
import numpy as np
import torch
import pyewts
import noisereduce as nr
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from num2tib.core import convert
from num2tib.core import convert2text
import re
from pathlib import Path
from IPython.display import Audio, display
import pyewts
import time
from statistics import mean
from tqdm.notebook import tqdm


from phonetic import text2phon

In [4]:
class Timer:
    def __enter__(self):
        # Record start time
        self.start_time = time.time()
        return self  # Return self to optionally access elapsed time later

    def __exit__(self, exc_type, exc_val, exc_tb):
        # Record end time and calculate elapsed time
        self.end_time = time.time()
        self.elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {self.elapsed_time:.4f} seconds")

In [5]:
Path.cwd()

PosixPath('/home/jovyan/speech_t5_tts/exp3/tests')

In [6]:
def get_model(model_name_or_path, cuda=True):
    model = SpeechT5ForTextToSpeech.from_pretrained(model_name_or_path)
    if cuda: model.to('cuda')
    return model

In [26]:
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

new_model = get_model("/data/volume/models/exp3/TTS_st5_phono_160k/checkpoint-84000")
old_model = get_model("openpecha/speecht5-tts-01")

In [27]:
converter = pyewts.pyewts()

def replace_numbers_with_convert(sentence, wylie=True):
    pattern = r'\d+(\.\d+)?'
    def replace(match):
        return convert(match.group(), wylie)
    result = re.sub(pattern, replace, sentence)

    return result

def cleanup_text(inputs):
    replacements = [
        ('_', '_'),
        ('*', 'v'),
        ('`', ';'),
        ('~', ','),
        ('+', ','),
        ('\\', ';'),
        ('|', ';'),
        ('╚',''),
        ('╗','')
    ]
    for src, dst in replacements:
        inputs = inputs.replace(src, dst)
    return inputs

def old_preprocessor(text):
    text = converter.toWylie(text)
    text = cleanup_text(text)
    print(text)
    text = replace_numbers_with_convert(text)
    return text

In [28]:
def replace_numbers_with_convert(sentence, wylie=True):
    pattern = r'\d+(\.\d+)?'
    def replace(match):
        return convert(match.group(), wylie)

    result = re.sub(pattern, replace, sentence)

    return result

def num2letter(sentence):
    tibetan_nums = "༠༡༢༣༤༥༦༧༨༩"
    for i, n in enumerate(tibetan_nums):
        sentence = sentence.replace(n, str(i))
    result = replace_numbers_with_convert(sentence, wylie=False)
    return result


def cleanup_text(text):
    replacements = [
        ('è', 'e'),  # Similar to "e" in "bed"
        ('ö', 'oe'), # Similar to "o" in "bird" or "u" in "burn" (British accent)
        ('ü', 'u'),  # Similar to "u" in "lune" or "y" in "myth" with rounded lips
        ('ɪ', 'i'),  # Similar to "i" in "bit"
        ('ʂ', 'sh'), # Similar to "sh" in "sheep" with retroflexion
        ('ḍ', 'd'),  # Retroflex "d" similar to hard "d"
        ('ṅ', 'ng'), # Similar to "ng" in "sing"
        ('ṭ', 't'),  # Retroflex "t" similar to hard "t"
    ]
    for src, dst in replacements:
        text = text.replace(src, dst)
    return text

def new_preprocessor(text):
    text = num2letter(text)
    text = text2phon(text)
    text = cleanup_text(text)
    return text

In [29]:
num2letter("ཀུན་གླེང་གསར་འགྱུར། ༢༠༢༣་ལོའི་ཟླ་༩ ཚེས་༢༧ །")

'ཀུན་གླེང་གསར་འགྱུར། གཉིས་སྟོང་ཉི་ཤུ་རྩ་གསུམ་་ལོའི་ཟླ་དགུ་ ཚེས་ཉི་ཤུ་རྩ་བདུན །'

In [30]:
def predict(text, model, preprocessor, cuda=True):
    if len(text.strip()) == 0:
        return (16000, np.zeros(0).astype(np.int16))
    text = preprocessor(text)
    inputs = processor(text=text, return_tensors="pt")
    input_ids = inputs["input_ids"]
    input_ids = input_ids[..., :model.config.max_text_positions]
    speaker_embedding = np.load("female_2.npy") - 0.003
    speaker_embedding = torch.tensor(speaker_embedding)
    if cuda:
        speech = model.generate_speech(input_ids.to('cuda'), speaker_embedding.to('cuda'), vocoder=vocoder.to('cuda')).to('cpu')
    else:
        speech = model.generate_speech(input_ids.to('cpu'), speaker_embedding.to('cpu'), vocoder=vocoder.to('cpu'))
    speech = nr.reduce_noise(y=speech, sr=16000)
    return (16000, speech)

In [31]:
examples = [
    # "ད་དེ་ཚོ་འདི་བྱེད་དགོས་རེད་ ན་ཚ་ མ་ཡོང་སྔོན་ལ་ཁོ་རང་ལ་ཡང་ཁྱི་ཁོ་རང་ཁོ་ལ་ཡང་ཁབ་རྒྱག་ཡ་ཡོད་རེད། ཨུན་སྔོན་འགོག་དང་རཱབྷིསས་ཁབ་རྒྱག་ཡ་ཡོད་རེད་ད།",
    "སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།",
    # "དངོས་གནས་ལབ་དགོས་རཱ་ད། མི་དབུལ་པོ་དེ་ཚོ་ལ་ག་རེ་ལབ་དགོས་རེད། སྦྱིན་པ་གཏང་ཡ་ཡོད་རཱ། ཨུན། དེ་འདྲ་གི་ལས་འགུལ་དེ་འདྲའི་མང་པོ་བརྩམས་ཀི་འདུག་བ། དེ་ཚོ་ཡང་ངས་ཚད་ལས་བརྒལ་བའི་ཡག་པོ་རེད་དྲན་གི་འདུག།",
    # "ཁོང་རྣམ་པ་ནི་སྤྱིར་བཏང་གི་གང་ཟག་ཅིག་མ་ཡིན་པར་མི་རབས་ནས་མི་རབས་རྒྱུད་པ་འཛིན་པའི་ནོར་བུ་ཡིན་ཞིང་།",
    # "ཨ་ལེ། སྤྱིར་བཏང་ད་ང་ཚོ་ད་ལྟ་ཁྱེད་རང་གིས་དམིགས་ཡུལ་ད་གལ་ཆེན་པོ་བརྩིས་ནས།",
    # "དེང་སྐབས་འཛམ་གླིང་ཁྱོན་ཡོངས་སུ་ཚན་རྩལ་འཕྲུལ་རིག་ཆེས་ཆེར་དར་ཞིང་། ལྷག་པར་ཉེ་བའི་ལོ་འདི་འགར་ཀུན་གྱིས་ཚ་ཚ་འུར་འུར་དུ་གླེང་བཞིན་པའི་མིས་བཟོས་རིག་ནུས་(Artificial intelligence)ཞེས་པ་བརྡ་འཕྲིན་འཕྲུལ་རིག་ཁྲོད་ཚད་གཞི་ཆེས་མཐོ་ཤོས་སུ་གྱུར་ཡོད་ལ།",
    # "འབྱུང་འགྱུར་འཕྲུལ་རིག་འདི་བརྒྱུད་ནས་ད་ལྟའི་འགྲོ་བ་མི་ཐུན་མོང་གི་ལོངས་སྤྱོད་དུ་གྱུར་པའི་ཤེས་ཡོན་མཐའ་དག་སྔར་ལས་ནུས་པ་ལྡན་པ་དང་། ལས་ཆོད་མཐོར་འདེགས་གཏོང་ཐུབ་པར་མ་ཟད། ང་ཚོ་བོད་མི་རིགས་ལྟ་བུའི་ཤེས་རིག་རྒྱ་ཆེ་ཡང་སྤྱོད་མཁན་གྲངས་ཉུང་བའི་རིགས་ལ་དམིགས་བསལ་ཕན་ཐོགས་རྒྱ་ཆེར་འབྱུང་ངེས།",
    # "དེ་བཞིན་གཞུང་ཚབ་ལར་སེན་ལགས་ཀྱིས། རྒྱལ་སྤྱིའི་སྤྱི་ཚོགས་ནས་རྒྱ་ནག་གཞུང་ལ་ཡང་ནས་ཡང་དུ་ཕྱི་གསལ་ནང་གསལ་ཡོང་བའི་རེ་སྐུལ་བྱས་ཀྱང་།"
]

In [34]:
for text in examples:
    print(text)
    print("Old Model".center(100))
    with Timer():
        rate, speech = predict(text, old_model, old_preprocessor)
    display(Audio(speech, rate=rate))
    print("New Model".center(100))
    with Timer():
        rate, speech = predict(text, new_model, new_preprocessor)
    display(Audio(speech, rate=rate))

སྟོབས་ཆེན་རྒྱལ་ཁབ་ཉི་ཤུའི་ལྷན་ཚོགས་ཐོག་ལ་རྒྱ་ནག་གཞུང་གིས་བོད་ནང་རིག་གཞུང་རྩ་གཏོར་ཀྱི་སྲིད་བྱུས་ཁག་དཔར་རིས་ཐོག་ནས་ལས་འགུལ་སྤེལ་བའི་སྐོར  འཇམ་དབྱངས་རྒྱ་མཚོ་ལགས་ཀྱིས་སྙན་སྒྲོན་གནང་གི་རེད།
                                             Old Model                                              
stobs chen rgyal khab nyi shu'i lhan tshogs thog la rgya nag gzhung gis bod nang rig gzhung rtsa gtor kyi srid byus khag dpar ris thog nas las 'gul spel ba'i skor__'jam dbyangs rgya mtsho lags kyis snyan sgron gnang gi red/


Elapsed time: 2.9351 seconds


                                             New Model                                              
Elapsed time: 2.6597 seconds


In [ ]:
def compare_model_performance(model1, model2, input_set, rounds=10):
    """
    Compares the inference performance of two models over multiple rounds.

    Parameters:
    - model1: The first model to evaluate.
    - model2: The second model to evaluate.
    - input_set: A list of inputs to use for inference.
    - rounds: Number of rounds to average inference time.

    Returns:
    - Dictionary with average inference time for each model.
    """

    def measure_inference_time(model, inputs, rounds):
        times = []
        for _ in tqdm(range(rounds)):
            start_time = time.time()
            for inp in tqdm(inputs):
                _ = model(inp)  # Run inference
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)
        return mean(times)

    # Measure inference time for model 1
    time_model1 = measure_inference_time(model1, input_set, rounds)

    # Measure inference time for model 2
    time_model2 = measure_inference_time(model2, input_set, rounds)

    # Return results
    return {
        "Model 1 Average Inference Time": time_model1,
        "Model 2 Average Inference Time": time_model2,
        "Faster Model": "Model 1" if time_model1 < time_model2 else "Model 2"
    }

In [55]:
def old_model_inference(text): return predict(text, old_model, old_preprocessor)
def new_model_inference(text): return predict(text, new_model, new_preprocessor)

In [56]:
compare_model_performance(old_model_inference, new_model_inference, examples, rounds=5)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

couldn't understand syllable རཱབྷིསས
couldn't understand syllable རཱབྷིསས
couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ


  0%|          | 0/8 [00:00<?, ?it/s]

couldn't understand syllable རཱབྷིསས
couldn't understand syllable རཱབྷིསས
couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ
couldn't understand syllable རཱ


{'Model 1 Average Inference Time': 19.22789978981018,
 'Model 2 Average Inference Time': 18.753835344314574,
 'Faster Model': 'Model 2'}